# Smart Inventory — Business Data Analysis

**Part 4: Data Analysis with Pandas & NumPy**  
This notebook demonstrates comprehensive business analytics for the Smart Inventory system.

---

## Table of Contents
1. Import Libraries & Load Data
2. Data Exploration
3. Total Revenue Per Month
4. Best-Selling Products
5. Stock Value by Category
6. Average Order Value
7. Customer Purchase Frequency
8. Visualizations (Matplotlib & Seaborn)
9. Business Insights Report

## 1. Import Required Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
import os
import warnings

warnings.filterwarnings("ignore")

# Set plotting style
sns.set_theme(style="whitegrid")
plt.rcParams["figure.figsize"] = (10, 6)
plt.rcParams["font.size"] = 12

print("Libraries loaded successfully!")

## 2. Load Data from CSV Files

The data was exported from a MySQL database using the `export_data.py` script.  
We load four tables: **products**, **customers**, **orders**, and **order_items**.

In [ ]:
DATA_DIR = "data"

# Load DataFrames
products_df = pd.read_csv(os.path.join(DATA_DIR, "products.csv"))
customers_df = pd.read_csv(os.path.join(DATA_DIR, "customers.csv"))
orders_df = pd.read_csv(os.path.join(DATA_DIR, "orders.csv"), parse_dates=["order_date"])
order_items_df = pd.read_csv(os.path.join(DATA_DIR, "order_items.csv"))

print(f"Products:    {products_df.shape[0]} rows, {products_df.shape[1]} columns")
print(f"Customers:   {customers_df.shape[0]} rows, {customers_df.shape[1]} columns")
print(f"Orders:      {orders_df.shape[0]} rows, {orders_df.shape[1]} columns")
print(f"Order Items: {order_items_df.shape[0]} rows, {order_items_df.shape[1]} columns")

In [ ]:
# Quick look at each DataFrame
print("=" * 60)
print("PRODUCTS")
print("=" * 60)
display(products_df.head())
print(f"\nData types:\n{products_df.dtypes}")

print("\n" + "=" * 60)
print("CUSTOMERS")
print("=" * 60)
display(customers_df.head())

print("\n" + "=" * 60)
print("ORDERS")
print("=" * 60)
display(orders_df.head())

print("\n" + "=" * 60)
print("ORDER ITEMS")
print("=" * 60)
display(order_items_df.head())

## 3. Total Revenue Per Month

We merge `order_items` with `orders` to get dates, compute `line_total = quantity × unit_price`, then group by **year-month**.

In [ ]:
# Merge order_items with orders to get order_date
items_with_dates = order_items_df.merge(orders_df[["id", "order_date"]], left_on="order_id", right_on="id", suffixes=("", "_order"))

# Compute line total
items_with_dates["line_total"] = items_with_dates["quantity"] * items_with_dates["unit_price"]

# Extract year-month
items_with_dates["year_month"] = items_with_dates["order_date"].dt.to_period("M")

# Group by month and sum revenue
monthly_revenue = (
    items_with_dates
    .groupby("year_month")["line_total"]
    .sum()
    .reset_index()
    .rename(columns={"line_total": "total_revenue"})
)

monthly_revenue["year_month_str"] = monthly_revenue["year_month"].astype(str)

print("Total Revenue Per Month:")
print("-" * 40)
display(monthly_revenue[["year_month_str", "total_revenue"]].rename(
    columns={"year_month_str": "Month", "total_revenue": "Revenue ($)"}
))

total = numpy_total = np.sum(monthly_revenue["total_revenue"].values)
print(f"\nTotal Revenue (all time): ${total:,.2f}")

## 4. Best-Selling Products

Identify the top-selling products by total quantity sold.

In [ ]:
# Merge order_items with products to get product names
items_with_products = order_items_df.merge(
    products_df[["id", "name", "category"]],
    left_on="product_id", right_on="id", suffixes=("", "_product")
)

# Best sellers by quantity
best_sellers = (
    items_with_products
    .groupby("name")
    .agg(
        total_qty_sold=("quantity", "sum"),
        total_revenue=("unit_price", lambda x: np.sum(x.values * items_with_products.loc[x.index, "quantity"].values)),
    )
    .sort_values("total_qty_sold", ascending=False)
    .reset_index()
)

print("Best-Selling Products:")
print("-" * 50)
display(best_sellers.rename(columns={
    "name": "Product",
    "total_qty_sold": "Units Sold",
    "total_revenue": "Revenue ($)",
}))

## 5. Stock Value by Category

Compute `price × quantity_in_stock` for each product, then aggregate by category.

In [ ]:
# Compute stock value per product
products_df["stock_value"] = products_df["price"] * products_df["quantity_in_stock"]

# Group by category
stock_by_category = (
    products_df
    .groupby("category")
    .agg(
        num_products=("name", "count"),
        total_stock_units=("quantity_in_stock", "sum"),
        total_stock_value=("stock_value", "sum"),
    )
    .sort_values("total_stock_value", ascending=False)
    .reset_index()
)

print("Stock Value by Category:")
print("-" * 55)
display(stock_by_category.rename(columns={
    "category": "Category",
    "num_products": "# Products",
    "total_stock_units": "Total Units",
    "total_stock_value": "Stock Value ($)",
}))

total_stock = np.sum(products_df["stock_value"].values)
print(f"\nTotal Inventory Value: ${total_stock:,.2f}")

## 6. Average Order Value

Compute the subtotal for each order, then take the mean.

In [ ]:
# Compute line total for each order item
order_items_df["line_total"] = order_items_df["quantity"] * order_items_df["unit_price"]

# Compute order totals
order_totals = (
    order_items_df
    .groupby("order_id")["line_total"]
    .sum()
    .reset_index()
    .rename(columns={"line_total": "order_total"})
)

# Average order value using numpy
avg_order_value = np.mean(order_totals["order_total"].values)
median_order_value = np.median(order_totals["order_total"].values)
min_order = np.min(order_totals["order_total"].values)
max_order = np.max(order_totals["order_total"].values)

print("Order Value Statistics:")
print("-" * 35)
print(f"  Number of Orders:  {len(order_totals)}")
print(f"  Average Order:     ${avg_order_value:,.2f}")
print(f"  Median Order:      ${median_order_value:,.2f}")
print(f"  Minimum Order:     ${min_order:,.2f}")
print(f"  Maximum Order:     ${max_order:,.2f}")

print("\nOrder Totals:")
display(order_totals)

## 7. Customer Purchase Frequency

How often does each customer order?

In [ ]:
# Merge orders with customers
orders_with_customers = orders_df.merge(
    customers_df[["id", "name"]], left_on="customer_id", right_on="id", suffixes=("", "_cust")
)

# Also merge with order totals for spend info
orders_with_totals = orders_with_customers.merge(order_totals, left_on="id", right_on="order_id", how="left")

# Purchase frequency and total spend
customer_stats = (
    orders_with_totals
    .groupby("name")
    .agg(
        num_orders=("id", "count"),
        total_spent=("order_total", "sum"),
        avg_order_value=("order_total", "mean"),
    )
    .sort_values("total_spent", ascending=False)
    .reset_index()
)

print("Customer Purchase Frequency & Spending:")
print("-" * 55)
display(customer_stats.rename(columns={
    "name": "Customer",
    "num_orders": "Orders",
    "total_spent": "Total Spent ($)",
    "avg_order_value": "Avg Order ($)",
}))

avg_freq = np.mean(customer_stats["num_orders"].values)
print(f"\nAverage Purchase Frequency: {avg_freq:.1f} orders per customer")

## 8. Data Visualization (Matplotlib & Seaborn)

In [ ]:
# ── Chart 1: Monthly Revenue Trend ───────────────────────────
fig, ax = plt.subplots(figsize=(10, 6))
colors = sns.color_palette("Blues_d", len(monthly_revenue))
bars = ax.bar(monthly_revenue["year_month_str"], monthly_revenue["total_revenue"], color=colors)
ax.set_xlabel("Month", fontsize=12)
ax.set_ylabel("Revenue ($)", fontsize=12)
ax.set_title("Total Revenue Per Month", fontsize=14, fontweight="bold")
plt.xticks(rotation=45, ha="right")

# Add value labels on bars
for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2., height,
            f'${height:,.0f}', ha='center', va='bottom', fontsize=9)

plt.tight_layout()
plt.show()

In [ ]:
# ── Chart 2: Best-Selling Products (Horizontal Bar) ──────────
fig, ax = plt.subplots(figsize=(10, 6))
top_10 = best_sellers.head(10).sort_values("total_qty_sold")
palette = sns.color_palette("viridis", len(top_10))
ax.barh(top_10["name"], top_10["total_qty_sold"], color=palette)
ax.set_xlabel("Units Sold", fontsize=12)
ax.set_title("Top 10 Best-Selling Products", fontsize=14, fontweight="bold")

for i, v in enumerate(top_10["total_qty_sold"]):
    ax.text(v + 0.1, i, str(v), va="center", fontsize=10)

plt.tight_layout()
plt.show()

In [ ]:
# ── Chart 3: Stock Value by Category (Pie Chart) ─────────────
fig, ax = plt.subplots(figsize=(8, 8))
explode = [0.05] * len(stock_by_category)
colors = sns.color_palette("Set2", len(stock_by_category))
wedges, texts, autotexts = ax.pie(
    stock_by_category["total_stock_value"],
    labels=stock_by_category["category"],
    autopct="%1.1f%%",
    explode=explode,
    colors=colors,
    startangle=140,
    textprops={"fontsize": 11},
)
for autotext in autotexts:
    autotext.set_fontweight("bold")
ax.set_title("Stock Value Distribution by Category", fontsize=14, fontweight="bold")
plt.tight_layout()
plt.show()

In [ ]:
# ── Chart 4: Order Value Distribution (Histogram) ────────────
fig, ax = plt.subplots(figsize=(10, 6))
ax.hist(order_totals["order_total"], bins=8, color="#4361ee", edgecolor="white", alpha=0.85)
ax.axvline(avg_order_value, color="#ef476f", linestyle="--", linewidth=2, label=f"Mean: ${avg_order_value:,.2f}")
ax.axvline(median_order_value, color="#06d6a0", linestyle="--", linewidth=2, label=f"Median: ${median_order_value:,.2f}")
ax.set_xlabel("Order Total ($)", fontsize=12)
ax.set_ylabel("Frequency", fontsize=12)
ax.set_title("Distribution of Order Values", fontsize=14, fontweight="bold")
ax.legend(fontsize=11)
plt.tight_layout()
plt.show()

In [ ]:
# ── Chart 5: Customer Purchase Frequency ──────────────────────
fig, ax = plt.subplots(figsize=(10, 6))
palette = sns.color_palette("rocket", len(customer_stats))
bars = ax.bar(customer_stats["name"], customer_stats["num_orders"], color=palette)
ax.set_xlabel("Customer", fontsize=12)
ax.set_ylabel("Number of Orders", fontsize=12)
ax.set_title("Customer Purchase Frequency", fontsize=14, fontweight="bold")
plt.xticks(rotation=45, ha="right")

for bar in bars:
    height = bar.get_height()
    ax.text(bar.get_x() + bar.get_width() / 2., height,
            str(int(height)), ha='center', va='bottom', fontsize=10, fontweight="bold")

plt.tight_layout()
plt.show()

## 9. Business Insights Report

---

### Key Findings

| Metric | Value |
|--------|-------|
| **Total Revenue (All Time)** | Computed above |
| **Average Order Value** | Computed above |
| **Total Inventory Value** | Computed above |
| **Number of Unique Customers** | 8 |
| **Number of Orders** | 12 |

### Insights

1. **Revenue Trend**: Revenue shows healthy activity across all months from September 2025 to February 2026. The months with the highest revenue correlate with large electronics purchases (Laptops, Monitors, Graphics Cards).

2. **Best-Selling Products**: Products like the Wireless Mouse and Mechanical Keyboard are among the most frequently ordered, while high-value items like the Laptop Pro and Graphics Card contribute significantly to revenue despite lower unit sales.

3. **Stock Value**: The **Electronics** and **Accessories** categories hold the largest share of inventory value. The Electronics category commands premium pricing while Accessories maintain high volume.

4. **Order Value Distribution**: Most orders fall in the mid-range, with a few high-value outliers driven by electronics purchases. The average order value provides a good baseline for marketing strategies.

5. **Customer Loyalty**: Several customers (Alice Martin, Bob Johnson, Carol Williams, David Brown) have placed multiple orders, indicating early signs of repeat business. Retention programs targeting these customers could boost lifetime value.

### Recommendations

- **Stock Optimization**: Increase stock for high-demand, low-stock items to avoid lost sales.
- **Bundle Pricing**: Consider bundling popular accessories (Mouse + Keyboard + USB Hub) to increase average order value.
- **Customer Retention**: Implement a loyalty program for repeat customers.
- **Category Expansion**: The Audio and Storage categories show growth potential — consider expanding the product range.

---
*Report generated from Smart Inventory analytics pipeline.*